In [1]:
import numpy as np
from numpy import random
import pandas as pd
from sklearn.neighbors import DistanceMetric
import functools
import operator
import matplotlib.pyplot as plt
import random
from scipy.spatial import distance

In [3]:
def simulate_data_5(N):
    X1 = np.random.uniform(low=0, high=25, size=(N))
    X2 = np.random.uniform(low=0, high=25, size=(N))
    X3 = np.random.uniform(low=0, high=25, size=(N))
    X4 = np.random.uniform(low=0, high=25, size=(N))
    X5 = np.random.uniform(low=0, high=25, size=(N))
    return X1, X2, X3, X4, X5 

In [5]:
from scipy.spatial import distance
def compute_dist(df):
    result = []
    for i in range(len(df)):
        for j in range(i+1,len(df)):
            d = distance.euclidean(df.iloc[i].to_list(),df.iloc[j].to_list())
            result.append([i, j, d])

    ordered_result = sorted(result, key=lambda t: t[::-1])
    return ordered_result

In [6]:
N = 200
X1, X2, X3, X4, X5 = simulate_data_5(N)
df = pd.DataFrame({'X1': X1, 'X2': X2,'X3': X3, 'X4': X4, 'X5': X5}, columns=['X1', 'X2','X3', 'X4', 'X5'])
ordered_result = compute_dist(df)

In [47]:
# Randomly assign indeces of cluster centroids:
def initiate_centroids(n, df):
    centroids = []
    random.seed(42) # use this since we want to compare the different number of clusters
    
    #generate random centroid indeces
    initial_index_centroid = random.sample(range(0, len(df)), n)
    
    #find the data points corresponding the the indeces:
    for i in initial_index_centroid:
        centroids.append(df.loc[i])
    return centroids


# To calculate the distance between two points:
def calc_distance(X1, X2):
    return(sum((X1 - X2)**2))**0.5


# To find the closest centroid to each data point:
def findClosestCentroids(centroids, df):
    assigned_centroid = []
    
    #iterate over every data point in the dataframe:
    for index, row in df.iterrows():
        distance=[]
        
        #find distance of data point with each cluster:
        for center in centroids:
            distance.append(calc_distance(row[:-1], center[:-1]))
            
        #assign data point to closest cluster:
        assigned_centroid.append(np.argmin(distance))
    return assigned_centroid


#To update the centroid of the clusters:
def calc_centroids(clusters, df):
    
    #initiate empty list for new centroids of each cluster:
    new_centroids = []
    
    #df including each point and its respective cluster
    new_df = pd.concat([pd.DataFrame(df), pd.DataFrame(clusters, columns=['cluster'])],
                      axis=1)
    
    #iterate over the distinct clusters
    for c in set(new_df['cluster']):
        
        #take out the data points corresponding to each cluster:
        current_cluster = new_df[new_df['cluster'] == c][new_df.columns[:-1]]
        
        #find the new cluster centroid which is the mean of the clusters we already assigned
        cluster_mean = current_cluster.mean(axis=0)
        
        #append the new centroid
        new_centroids.append(cluster_mean)
        
    return new_centroids


#Recursively find and update cluster centroids:
#n: number of clusters, df: dataframe of data points, iterations: number of iterations
def recursive_centroid_find(n, df):
    
    #initiate centroids:
    old_centroids = initiate_centroids(n, df)
  
    #initiate new centroids (do this for first while loop condition to work)
    new_centroids = calc_centroids(findClosestCentroids(old_centroids, df), df)

    # Iterate until the absoulte difference between the coordinates of all centroids 
    #does not change, ie: the k-means algorithm converges:
    while np.absolute(np.array(new_centroids) - np.array(old_centroids)).all() > 0.005:
    
        #set new centroids as the old ones:
        old_centroids = new_centroids
        
        #find the new ones:
        new_centroids = calc_centroids(findClosestCentroids(old_centroids, df), df)
        
        #print(np.array(new_centroids))
        #plot the centroids after every iteration:
        #plt.figure()
        #plt.scatter(np.array(centroids)[:, 0], np.array(centroids)[:, 1], color='red')
        #plt.scatter(df.X1, df.X2, alpha=0.1)
        #plt.show()
    return pd.DataFrame(new_centroids)

In [48]:
%time
recursive_centroid_find(4, df)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16.5 µs


,X1,X2,X3,X4,X5
0,19.324653,8.368270,12.804679,6.891333,14.256500
1,11.664595,8.089540,5.380657,19.998803,13.211084
2,10.776673,18.884031,12.016986,9.103242,10.803905
3,6.026751,7.183992,17.947539,16.665132,13.434881
